<a href="https://colab.research.google.com/github/yufang934/deeplearning/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="", 
    origin="http://", 
    extract=True,
)

84131840/84125825 [==============================] - 4s 0us/step


In [ ]:
import glob
#glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd
def read(path):
  with open(path, "r", encoding="utf-8") as f:
    content = f.read()
  return content
def get_data(t):
  dn = os.path.dirname(dataset)
  pattern = os.path.join(dn, "aclImdb","train","pos","*")
  pos = glob.glob(pattern)
  pattern = os.path.join(dn, "aclImdb","train","neg","*")
  neg = glob.glob(pattern)
  sentiments = [1] * len(pos) + [0] * len(neg)
  contents = map(read, pos + neg)
  df = pd.DataFrame({
      "contents":contents,
      "sentiment":sentiments
  })
  return df
train_df = get_data("train")
test_df = get_data("test")
test_df

,contents,sentiment
0,The movie is great and I like the story. I pre...,1
1,This is of of Sammo's great early comedy films...,1
2,"As a long time Red Sox fan, I just had to go s...",1
3,This movie awed me so much that I watch it at ...,1
4,This adaptation of M.R. James's short story 'A...,1
...,...,...
24995,It is unbelievable that a script as cliché and...,0
24996,A film like Amazing Grace and Chuck is a perfe...,0
24997,This is the biggest load of crap that I have s...,0
24998,"I awake suddenly, aware that I'm drooling onto...",0


In [ ]:
#文本預處理(補0值)
#https://keras.io/zh/preprocessing/text/
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [ ]:
# tok.word_index
tok.index_word[2]

'and'

In [ ]:
# pd.DataFrame(train_df_seq)
# truncating: pre(刪前面) 12345 -> 45 ,post(刪後面) 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
#每個文章取512個詞
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,17,6,84,2,10,37,1,62,10,2781,11,17,71,82,17,138,1,2753,1189,17,2,705,17,10,40,373,1,879,10,37,3,632,2,79,3,280,62,7,7
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,158,33,25,98,144,325,81,8,2576,1983,8,70,10,479,204,107,192,425,1211,14,605,1614,81,8,151,296,99,96,7,7,11,6,28,6,15,98,39,1474,844,334
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,76,5,810,1624,6,639,3,212,64,45,22,23,264,15,3,17,5,103,16,126,1301,39,2665,82,258,45,22,23,3,2353,334,2,57,50,258,45,22,23,764,334
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,157,2,254,82,15,125,39,430,1755,119,3,125,289,4,3,746,437,8,1,390,4,632,71,19,259,10,255,95,196,1768,8,65,203,93,11,19,6,3,212,64
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,71,1670,231,9,97,25,410,3,368,84,1250,14,9,6,9,29,761,3,114,2,3,224,50,5,267,1,1306,59,25,74,2373,7,7,10,199,9,690,43,4,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,18,89,448,1,94,65,903,14,70,1,862,274,77,126,122,45,22,116,39,141,24,2150,5,2220,43,57,50,10,654,26,97,199,69,145,104,631,4,58,110,142
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2770,34,1072,11,7,7,1,19,6,287,61,238,4,155,379,7,7,45,22,178,5,64,3,84,19,41,1,4,322,1227,16,881,302,7,7,440,22,23,54,1260
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,44,573,704,113,2,6,52,361,349,58,461,2,10,848,676,122,1,17,100,41,317,32,531,72,66,801,5,199,9,3,577,18,161,97,604,11,78,21,103,9
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,53,3,168,669,68,457,8,1,67,2,26,56,12,234,29,1,795,77,27,2356,54,28,77,147,86,370,9,6,2,26,56,67,398,1,1779,7,7,871,74,2808


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
    #tokenize的時候多加了"0"，故是3000(token)+1(padding)=3001種
    #param = 3001 * 128
    Embedding(3001, 128, mask_zero=True, input_length=512),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]

model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8388736   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 258       
Total params: 8,773,122
Trainable params: 8,773,122
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [ ]:
# epoch: 整份資料看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)
#val_loss過擬合跑太快(0.3->0.6->0.78->0.84)

Epoch 1/50
225/225 - 5s - loss: 0.6063 - accuracy: 0.6442 - val_loss: 0.3972 - val_accuracy: 0.8192
Epoch 2/50
225/225 - 4s - loss: 0.2609 - accuracy: 0.8977 - val_loss: 0.3018 - val_accuracy: 0.8740
Epoch 3/50
225/225 - 4s - loss: 0.0977 - accuracy: 0.9680 - val_loss: 0.3604 - val_accuracy: 0.8752
Epoch 4/50
225/225 - 4s - loss: 0.0180 - accuracy: 0.9958 - val_loss: 0.6684 - val_accuracy: 0.8340
Epoch 5/50
225/225 - 4s - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.7897 - val_accuracy: 0.8376
Epoch 6/50
225/225 - 4s - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.8485 - val_accuracy: 0.8384
Epoch 7/50
225/225 - 4s - loss: 5.4784e-04 - accuracy: 1.0000 - val_loss: 1.0406 - val_accuracy: 0.8268


In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.1282 - accuracy: 0.9608


[0.12822382152080536, 0.9607999920845032]